# [Asynchronous iteration](https://exploringjs.com/impatient-js/ch_async-iteration.html)

## Basic asynchronous iteration

### Protocol: async iteration

* an _iterable_ is a data structure whose contents can be accessed via iteration
    - it is a factory for iterators
* an _iterator_ is a factory for iteration results that we retrieve by calling the method .next()
* each _iterationResult_ contains the iterated .value and a boolean .done that is true after the last element and false before
***
* for the protocol for asynchronous iteration, we only want to change one thing:
    - the values produced by .next() should be delivered asynchronously
* there are 2 options:
    1. .value could contain a Promise<T>
    2. .next() could return Promise<IteratorResult<T>>
* so do we wrap the value or the whole iterator result in a Promise?
    - we have to choose option 2
    - reason being, when .next() returns a result, it starts an asynchronous computation and whether that computation produces a value or signals the end of the iteration can only be determined after it is finished
    - therefore, both .done and .value need to be wrapped in a Promise

In [ ]:
// interfaces for synchronous iteration

interface Iterable<T> {
    [Symbol.iterator](): Iterator<T>;
}

interface Iterator<T> {
    next(): IteratorResult<T>;
}

interface IteratorResult<T> {
    value: T;
    done: boolean;
}

In [ ]:
// interfaces for asynchronous iteration
// the only difference to the synchronous iterface is the return type of .next() (line A)

interface AsyncIterable<T> {
    [Symbol.asyncIterator](): AsyncIterator<T>;
}

interface AsyncIterator<T> {
    next(): Promise<IteratorResult<T>>; // A
}

interface IteratorResult<T> {
    value: T;
    done: boolean;
}

### Using async iteration directly

* line A: create an asynchronous iterable over the value 'a' and 'b'
* we call .next() is line B, C, and D
    - each time, we use .then() to unwrap the Promise and assert.deepEqual() to check the unwrapped value

In [ ]:
const asyncIterable = syncToAsyncIterable(['a','b']); // A
const asyncIterator = asyncIterable[Symbol.asyncIterator]();

// call .next() until .done is true
asyncIterator.next() // B
    .then(iteratorResult => {
        assert.deepEqual(
            iteratorResult,
            { value: 'a', done: false}
        );
        return asyncIterator.next(); // C
    })
    .then(iteratorResult => {
        assert.deepEqual(
            iteratorResult,
            { value: 'b', done: false}
        );
        return asyncIterator.next(); // D
    })
    .then(iteratorResult => {
        assert.deepEqual(
            iteratorResult,
            { value: undefined , done: true}
        );
        return asyncIterator.next(); // C
    })

* we can simplify the code using an async function and unwrap the Promises via await

In [ ]:
const asyncIterable = syncToAsyncIterable(['a','b']); // A
const asyncIterator = asyncIterable[Symbol.asyncIterator]();

assert.deepEqual(
    await asyncIterator.next(),
    { value: 'a', done: false }
)

assert.deepEqual(
    await asyncIterator.next(),
    { value: 'b', done: false }
)

assert.deepEqual(
    await asyncIterator.next(),
    { value: undefined, done: true }
)

### Using async iteration via for-await-of

* asynchronous iteration protocol is not meant to be used directly
* for-await-of is an asynchronous version of for-of and can be used in async functions and async generators

In [ ]:
for await (const x of syncToAsyncIterable(['a', 'b'])) {
    console.log(x);
}

// Output:
// 'a'
// 'b'

In [2]:
// it is flexible and can also support synchronous iterables
async function syncFor() {
    for await (const x of ['a', 'b']) {
        console.log(x);
    }  
}

syncFor();

Promise { <pending> }

a
b


In [3]:
// also able to support synchronous iterables over values that are wrapped in Promises
async function syncFor() {
    const arr = [Promise.resolve('a'), Promise.resolve('b')];
    for await (const x of arr) {
        console.log(x);
    }  
}

syncFor();

Promise { <pending> }

a
b


## Asynchronous Generators

* an asynchronous generator is 2 things at the same time:
    - an async function (input): we can use await and for-await-of to retrieve data
    - a generator that returns an asynchronous iterable (output): we can use yield and yield* to produce data
* therefore, an asynchronous generator has:
    - Input that can be:
        * synchronous: single values, sync iterables
        * asynchronous: Promises, async iterables
    - Output that is an asynchronous iterable

In [ ]:
async function* asyncGen() {
    // Input: Promises, async iterables
    const x = await somePromise;
    for await (const y of someAsyncIterable) {
        // ...
    }
    
    // Output
    yield someValue;
    yield* otherAsyncGen();
}